# test for attentino block

In [1]:
from dataset.dataset import *
from PIL import Image
import matplotlib.pyplot as plt

test_ds = RealEstate_dataset(info_root = 'dataset/RealEstate10K', data_root = 'dataset/RealEstate_data', mode = 'test')
img_s, img_t, K_s, R_rel, t_rel = test_ds[1]


from models.epipolar_unet import *

W_Mat = epipolar_weight_Mat((64, 64), K_s, R_rel, t_rel)

#print(W_Mat.shape)

attn_resolution = 32
query_channels = 256
key_channels = 1
t = torch.tensor([1000])
source_input = torch.randn(1, 1, 128, 128)
Unet_input = torch.randn(1, query_channels, attn_resolution, attn_resolution)

e_attn = epipolar_Attention_Block(num_heads = 4, query_channels=query_channels, 
                                  key_channels=key_channels, attn_resolution=attn_resolution)


res = e_attn(source_input, Unet_input, t, W_Mat)

print(res.shape)

c:\Users\Administrator\anaconda3\envs\zero123\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# test for whole model

In [6]:
keys = {'image_size': 64, 'num_channels': 192, 
        'num_res_blocks': 3, 'num_heads': 4, 'num_heads_upsample': -1, 
        'num_head_channels': 64, 'attention_resolutions': '32,16,8', 'channel_mult': '', 'dropout': 0.1, 
        'class_cond': True, 'use_checkpoint': False, 'use_scale_shift_norm': True, 
        'resblock_updown': True, 'use_fp16': False, 'use_new_attention_order': True, 
        'learn_sigma': True, 'diffusion_steps': 1000, 'noise_schedule': 'cosine', 
        'timestep_respacing': '250', 'use_kl': False, 'predict_xstart': False, 'rescale_timesteps': False, 
        'rescale_learned_sigmas': False, 'feature_channels': 1, 'epipolar_distance_threshold': 0.1}
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = create_epipolar_model(**keys)

    
model.load_state_dict(torch.load('weight/pretrain/openai/64x64_diffusion.pt'), strict=False)
model = model.to(device)
for param in model.parameters():
    param.to(device)
    
x = torch.randn(1, 3, 64, 64)
t = torch.ones(1, dtype=torch.float32) * 1000
weight_Mat = torch.randn(1, 64*64, 64*64)
f = torch.randn(1, 1, 64, 64)

#for name, param in model.named_parameters():
#    
#    print(f"Parameter '{name}' is on device: {param.device}")


x = x.to(device)
t = t.to(device)
weight_Mat = weight_Mat.to(device)
f = f.to(device)

res = model(x, t, f, weight_Mat)
print(res.shape)


<All keys matched successfully>

# Test for pipeline

In [1]:
from models.epipolar_unet import *

from models.unet import *
from utils.script_util import *
from dataset.dataset import *

#MODEL_FLAGS="--attention_resolutions 32,16,8 --class_cond True --diffusion_steps 1000 --dropout 0.1 --image_size 64 --learn_sigma True --noise_schedule cosine --num_channels 192 
from dataset.dataset import *
from PIL import Image
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import matplotlib.pyplot as plt


test_ds = RealEstate_dataset(info_root = 'dataset/RealEstate10K', data_root = 'dataset/RealEstate_data', mode = 'test')
loader = DataLoader(test_ds, batch_size=1, num_workers=0, pin_memory=True,
                                drop_last=True)
# for idx, data in enumerate(loader):
#     print(idx, end='\r')
    
# prepare source view encoder
model_type = "DPT_Large" 
midas = torch.hub.load("intel-isl/MiDaS", model_type)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
midas.to(device)
midas_transforms = torch.hub.load("intel-isl/MiDaS", "transforms")
if model_type == "DPT_Large" or model_type == "DPT_Hybrid":
    transform = midas_transforms.dpt_transform
else:
    transform = midas_transforms.small_transform
    
# fetch data
example = None
for idx, data in enumerate(loader):
    example = data
    break

img_s, img_t, K_s, R_rel, t_rel = example

# print(torch.min(img_s))
img_s = img_s.cuda()
# compute depth
feature = midas(img_s)  

c:\Users\Administrator\anaconda3\envs\zero123\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'utils.script_utils'

In [ ]:
from models.epipolar_unet import *

from models.unet import *
from utils.script_util import *

from dataset.dataset import *

#MODEL_FLAGS="--attention_resolutions 32,16,8 --class_cond True --diffusion_steps 1000 --dropout 0.1 --image_size 64 --learn_sigma True --noise_schedule cosine --num_channels 192 
from dataset.dataset import *
from PIL import Image
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import matplotlib.pyplot as plt



# print(feature.shape)from utils.script_utils import *
from utils.resample import *

keys = {'image_size': 64, 'num_channels': 192, 
        'num_res_blocks': 3, 'num_heads': 4, 'num_heads_upsample': -1, 
        'num_head_channels': 64, 'attention_resolutions': '32,16,8', 'channel_mult': '', 'dropout': 0.1, 
        'class_cond': True, 'use_checkpoint': False, 'use_scale_shift_norm': True, 
        'resblock_updown': True, 'use_fp16': True, 'use_new_attention_order': True, 
        'learn_sigma': True, 'diffusion_steps': 1000, 'noise_schedule': 'cosine',
        'timestep_respacing': '250', 'use_kl': False, 'predict_xstart': False, 'rescale_timesteps': False, 
        'rescale_learned_sigmas': False, 'use_epipolar': True, 'schedule_sampler': 'uniform', 'num_epochs': 1000}
    
model, diffusion = create_model_and_diffusion(**keys)
schedule_sampler = create_named_schedule_sampler(keys['schedule_sampler'], diffusion)


In [1]:

from utils.dataset import *
from PIL import Image
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader

test_ds = RealEstate_dataset(info_root = 'dataset/RealEstate10K', data_root = 'dataset/RealEstate_data', mode = 'test')
loader = DataLoader(test_ds, batch_size=2, shuffle=False, num_workers=1, drop_last=True)

c:\Users\Administrator\anaconda3\envs\zero123\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
example = None
for idx, data in enumerate(loader):
    example = data 
    break


In [3]:
img_s, img_t, K_s, R_rel, t_rel = example

print(K_s.shape)

torch.Size([2, 3, 3])


In [7]:
from models.epipolar_unet import *
from models.epipolar_unet import *

from models.unet import *
# from utils.script_util import *

#MODEL_FLAGS="--attention_resolutions 32,16,8 --class_cond True --diffusion_steps 1000 --dropout 0.1 --image_size 64 --learn_sigma True --noise_schedule cosine --num_channels 192 
from utils.dataset import *
from PIL import Image
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import utils.logger as logger
# from utils.train_util import *
from utils.script_util import *

# print(feature.shape)from utils.script_utils import *
from utils.resample import *

keys = {'image_size': 64, 'num_channels': 192, 
        'num_res_blocks': 3, 'num_heads': 4, 'num_heads_upsample': -1, 
        'num_head_channels': 64, 'attention_resolutions': '32,16,8', 'channel_mult': '', 'dropout': 0.1, 
        'class_cond': True, 'use_checkpoint': False, 'use_scale_shift_norm': True, 
        'resblock_updown': True, 'use_fp16': False, 'use_new_attention_order': True, 
        'learn_sigma': True, 'diffusion_steps': 1000, 'noise_schedule': 'cosine',
        'timestep_respacing': '250', 'use_kl': False, 'predict_xstart': False, 'rescale_timesteps': False, 
        'rescale_learned_sigmas': False, 'use_epipolar': True}

sample_args = {'schedule_sampler': 'uniform', 'num_epochs': 1000}

model, diffusion = create_model_and_diffusion(**keys)


In [8]:
feature = torch.randn(1, 1, 64, 64).cuda()
W_Mat = torch.randn(1, 64*64, 64*64).cuda()
x = torch.randn(1, 3, 64, 64).cuda()
t = torch.ones(1, dtype=torch.float32) * 1000
t = t.cuda()
model = model.cuda()
model_kwargs = {'f' : feature, 'Weight_Mat': W_Mat}

res = model(x, t, **model_kwargs)
